In [1]:
# Run this cell just once! (or restart Kernel before second time)

import os
os.chdir('..')

In [2]:
import fastText
import re
import pandas as pd
from utils.fixed import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
TRAINING_DATA_PATH = os.path.join(MAIN_PATH, DATA_PATH, 'sentiment_m140_.csv')
TRAIN_FILE_PATH = os.path.join(MAIN_PATH, DATA_PATH, 'test-doc.txt')
TEST_FILE_PATH = os.path.join(MAIN_PATH, DATA_PATH, 'train-doc.txt')

In [4]:
SLANG_DICT=load_slang(SLANG_PATH)
train_df = pd.read_csv(TRAINING_DATA_PATH,sep=',',encoding = "ISO-8859-1", lineterminator='\n',header=0)
train_df.columns=['sentiment','id','date','query','user','tweetText']

### PREPARE DATA

In [5]:
X, y = get_processed_tweets(train_df,SLANG_DICT,False,False,False,True)
indexes = list(train_df.index)
for i in indexes:
    X[i] = X[i]+' '+'__label__'+str(y[i])

In [6]:
X_train, X_test, y_train, y_test,index_train,index_test = train_test_split(X,y,indexes,test_size=0.5, random_state=42)

In [ ]:
with open(TRAIN_FILE_PATH, mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(X_train))
    myfile.write('\n')
with open(TEST_FILE_PATH, mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(X_test))
    myfile.write('\n')

### TRAIN THE MODEL

In [ ]:
vector_size = 500
ngrams = 2
clf = fastText.train_supervised(TRAIN_FILE_PATH,wordNgrams=ngrams,dim=vector_size,minCount=1)

### SAVE THE MODEL

In [ ]:
FASTTEXT_MODEL_PATH = os.path.join(MAIN_PATH,MODEL_PATH,'fasttext-'+str(vector_size)+'.model')
clf.save_model(FASTTEXT_MODEL_PATH)

# To load the model run: 
# model=fastText.load_model('fasttext.model')

### CALCULATING TRAIN SET ACCURACY

In [ ]:
train_list = list(train_df.loc[[i for i in index_train], ['text']]['text'])
result = clf.predict(train_list)
labels_train=[int(result[0][i][0].replace('__label__','')) for i in range(len(result[0]))]
accuracy_score(labels_train, y_train)

### CALCULATING TEST SET ACCURACY

In [ ]:
test_list = list(train_df.loc[[i for i in index_test], ['text']]['text'])
result = clf.predict(test_list)
labels_test=[int(result[0][i][0].replace('__label__','')) for i in range(len(result[0]))]
accuracy_score(labels_test, y_test)